In [1]:
import pandas as pd
import snorkel
import matplotlib.pyplot as plt
from snorkel.labeling import labeling_function
from snorkel.labeling import LFAnalysis
from snorkel.labeling.model import MajorityLabelVoter
from snorkel.labeling import PandasLFApplier
from snorkel.labeling.model import LabelModel
import seaborn as sns

/Users/chuli/opt/anaconda3/envs/PSML/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Reading the data

In [2]:
#readin the curb_fp.csv file
curb_fp = pd.read_csv('data/curb_fp.csv')
curb_fp

,audit_task_id,label_id,label_type,severity,correct,high_quality_user,gsv_panorama_id,agree_count,disagree_count,notsure_count,...,canvas_width,canvas_height,canvas_x,canvas_y,zoom,heading,pitch,photographer_heading,photographer_pitch,geometry
0,18927,85882,CurbRamp,1.0,NaN,False,yLFcq-aaf-Ag_iITWsAWnQ,0,0,0,...,720,480,169,141,1,34.671875,-24.8750,90.021843,1.101685,POINT (1275705.9782665605 214304.255489019)
1,19027,86190,CurbRamp,1.0,1.0,False,4vGkAd8TMCx4FgyoDV7Pmw,1,0,0,...,720,480,305,283,1,270.000000,-10.0000,90.147316,-9.087791,POINT (1261597.9105232325 237078.0516430551)
2,19143,86478,CurbRamp,1.0,NaN,False,JXW_FToo2iFIB0NslFrCeg,0,0,0,...,720,480,193,153,1,195.500000,-17.8750,350.722260,2.929657,POINT (1272276.4832222403 258029.08524980946)
3,19411,86767,CurbRamp,1.0,NaN,False,Z1u8iq5TEuS8-n_kZKRc8A,0,0,0,...,720,480,526,315,1,174.437500,-5.8750,156.979935,-0.590858,POINT (1283388.097714087 258754.43556004378)
4,19411,86768,CurbRamp,1.0,1.0,False,Z1u8iq5TEuS8-n_kZKRc8A,1,0,0,...,720,480,285,212,1,128.500000,-13.0000,156.979935,-0.590858,POINT (1283444.4740236972 258754.76484089182)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10760,18558,84922,CurbRamp,NaN,NaN,False,-B8963iDbur4NYIYzdTJXA,0,0,0,...,720,480,505,230,1,324.500000,-17.8125,220.064758,-4.507965,POINT (1269212.1309289748 227124.1054278093)
10761,75784,230293,CurbRamp,1.0,NaN,True,3KIM9a467DImsFzpQ9xAqQ,0,0,0,...,720,480,449,120,3,169.227676,-15.8750,22.459303,2.169060,POINT (1280570.7039149986 206540.29524511957)
10762,76193,231338,CurbRamp,1.0,NaN,True,d3oBrJ-nDtKMwlE7svc6pg,0,0,1,...,720,480,267,187,1,253.562500,-35.0000,177.959915,-0.962440,POINT (1282827.7064719573 208828.7977988457)
10763,76193,231339,CurbRamp,1.0,NaN,True,AOIBJ12BjFanDJF1Ark05Q,0,0,0,...,720,480,516,171,1,110.312500,-35.0000,179.365723,-1.093400,POINT (1282862.9675030417 208900.50316090064)


In [3]:
#put curb_fp[label_id] into a list
curb_fp_ids = curb_fp['label_id'].tolist()

In [4]:
labels = pd.read_csv('data/labels.csv')

In [5]:
labels

,audit_task_id,label_id,label_type,severity,correct,high_quality_user,gsv_panorama_id,agree_count,disagree_count,notsure_count,...,heading,pitch,photographer_heading,photographer_pitch,label_tags,label_description,heading_diff,pitch_diff,geometry,clustered
0,18604,85055,CurbRamp,1.0,NaN,False,NjPAkwTxWsayAq3kCugvdA,0,0,0,...,107.750000,-15.62500,270.556610,0.907036,0.0,0.0,-162.806610,-16.532036,POINT (-122.3327865600586 47.61859893798828),-1
1,18608,85057,Obstacle,2.0,1.0,False,f_Mibt-6nJU7HTN7kx5ZGQ,2,0,0,...,38.375000,-21.87500,164.133438,0.335564,0.0,0.0,-125.758438,-22.210564,POINT (-122.3318862915039 47.61780548095703),-1
2,18608,85059,CurbRamp,1.0,NaN,False,f_Mibt-6nJU7HTN7kx5ZGQ,0,0,0,...,131.187500,-24.50000,164.133438,0.335564,0.0,0.0,-32.945938,-24.835564,POINT (-122.33183288574219 47.61751174926758),-1
3,18608,85060,CurbRamp,1.0,NaN,False,f_Mibt-6nJU7HTN7kx5ZGQ,0,0,0,...,131.187500,-24.50000,164.133438,0.335564,0.0,0.0,-32.945938,-24.835564,POINT (-122.33187866210938 47.61751937866211),-1
4,18608,85062,CurbRamp,1.0,NaN,False,f_Mibt-6nJU7HTN7kx5ZGQ,0,0,0,...,257.187500,-35.00000,164.133438,0.335564,0.0,0.0,93.054062,-35.335564,POINT (-122.33204650878906 47.617637634277344),-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196809,76181,231270,CurbRamp,1.0,NaN,True,R-sTl5O6sOdVPurgTWy3EQ,0,0,0,...,332.468750,-35.00000,215.130295,-0.157570,0.0,0.0,117.338455,-34.842430,POINT (-122.26725769042969 47.540771484375),-1
196810,76183,231285,NoSidewalk,5.0,NaN,True,N5MrAAtHZO2JGXUrYAYnSQ,0,0,0,...,0.890625,-11.65625,179.161057,-5.825714,0.0,0.0,-178.270432,-5.830536,POINT (-122.26344299316406 47.55229187011719),-1
196811,76193,231338,CurbRamp,1.0,NaN,True,d3oBrJ-nDtKMwlE7svc6pg,0,0,1,...,253.562500,-35.00000,177.959915,-0.962440,0.0,0.0,75.602585,-34.037560,POINT (-122.28179168701172 47.563270568847656),-1
196812,76193,231339,CurbRamp,1.0,NaN,True,AOIBJ12BjFanDJF1Ark05Q,0,0,0,...,110.312500,-35.00000,179.365723,-1.093400,0.0,0.0,-69.053223,-33.906600,POINT (-122.28165435791016 47.56346893310547),-1


## Labeling functions based on heuristics

In [6]:
# Define the label mappings for convenience
NOT_SURE = -1
WRONG = 0
CORRECT = 1

### intersection

In [7]:
#check if it is in the curb false positive list
@labeling_function()
def intersection(x):
    # x in this case is a row of the dataframe
    return WRONG if x["label_id"] in curb_fp_ids else NOT_SURE

### cluster

In [ ]:
#check if the label is in the cluster
@labeling_function()
def clustered(x):
    # x in this case is a row of the dataframe
    return CORRECT if x['clustered']  == 1 else NOT_SURE

### severity

In [8]:
#check if it is a severity 4 or higher
@labeling_function()
def severity(x):
    # x in this case is a row of the dataframe
    return CORRECT if x["severity"] >= 4 else NOT_SURE

### gsv

In [9]:
#check if the user zoomed in
@labeling_function()
def zoom(x):
    # x in this case is a row of the dataframe
    if x["zoom"] > 2:
        return CORRECT
    elif x["zoom"] == 1:
        return WRONG
    else:
        return NOT_SURE

In [10]:
#check if the user changed the heading
@labeling_function()
def heading(x):
    # x in this case is a row of the dataframe
    return CORRECT if abs(x['heading_diff'])  >= 100 else NOT_SURE

In [11]:
#check if the user changed the pitch
@labeling_function()
def pitch(x):
    # x in this case is a row of the dataframe
    return CORRECT if abs(x['pitch_diff'])  >= 20 else NOT_SURE

### optional input

In [12]:
#check if the user has put a tag
@labeling_function()
def tags(x):
    # x in this case is a row of the dataframe
    return CORRECT if x['label_tags']  == 1 else NOT_SURE

In [13]:
#check if the user has put a comment
@labeling_function()
def description(x):
    # x in this case is a row of the dataframe
    return CORRECT if x['label_description']  == 1 else NOT_SURE

In [15]:
#check if the user has put a tag
@labeling_function()
def missing(x):
    # x in this case is a row of the dataframe
    return CORRECT if x['label_type']  == 'NoSidewalk' else NOT_SURE

In [16]:
lfs = [
    intersection, 
    zoom, 
    tags, 
    description,
    severity,
    pitch,
    heading,
    clustered
    ]

In [17]:
applier = PandasLFApplier(lfs=lfs)

In [18]:
#read in the labels.csv file
df_train = pd.read_csv('data/labels.csv')

In [19]:
# read in gt.csv file
df_test = pd.read_csv('data/gt_3.csv')
Y_test = df_test.verified.values


In [20]:
L_train = applier.apply(df=df_train)
L_test = applier.apply(df=df_test)

100%|██████████| 16890/16890 [00:01<00:00, 8630.22it/s]


In [21]:
LFAnalysis(L=L_train, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
intersection,0,[0],0.054696,0.051511,0.046379
zoom,1,"[0, 1]",0.782612,0.673992,0.561769
tags,2,[1],0.081829,0.081793,0.059823
description,3,[1],0.001468,0.001468,0.000945
severity,4,[1],0.278065,0.266124,0.205621
pitch,5,[1],0.523931,0.491434,0.411160
heading,6,[1],0.521030,0.471943,0.330739
clustered,7,[1],0.033367,0.033265,0.021264


In [22]:
majority_model = MajorityLabelVoter()
preds_train = majority_model.predict(L=L_train)

In [23]:
label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train=L_train, n_epochs=500, log_freq=100, seed=123)

INFO:root:Computing O...
INFO:root:Estimating \mu...
  0%|          | 0/500 [00:00<?, ?epoch/s]INFO:root:[0 epochs]: TRAIN:[loss=0.990]
INFO:root:[100 epochs]: TRAIN:[loss=0.020]
INFO:root:[200 epochs]: TRAIN:[loss=0.015]
 48%|████▊     | 240/500 [00:00<00:00, 2399.73epoch/s]INFO:root:[300 epochs]: TRAIN:[loss=0.012]
INFO:root:[400 epochs]: TRAIN:[loss=0.012]
100%|██████████| 500/500 [00:00<00:00, 2924.49epoch/s]
INFO:root:Finished Training


In [24]:
majority_acc = majority_model.score(L=L_test, Y=Y_test, tie_break_policy="random")[
    "accuracy"
]
print(f"{'Majority Vote Accuracy:':<25} {majority_acc * 100:.1f}%")

label_model_acc = label_model.score(L=L_test, Y=Y_test, tie_break_policy="random")[
    "accuracy"
]
print(f"{'Label Model Accuracy:':<25} {label_model_acc * 100:.1f}%")

Majority Vote Accuracy:   75.0%
Label Model Accuracy:     73.8%


In [25]:
df_test[df_test['verified']==1]

,verified,label_id,audit_task_id,label_type,severity,correct,high_quality_user,gsv_panorama_id,agree_count,disagree_count,...,heading,pitch,photographer_heading,photographer_pitch,label_tags,label_description,heading_diff,pitch_diff,geometry,clustered
0,1,86788,19418,SurfaceProblem,2.0,1.0,False,BpVD1H9f1A6URlrjPUcyZg,2,0,...,183.062500,-7.125000,358.678406,6.516869,0.0,0.0,-175.615906,-13.641869,POINT (-122.28125 47.698307037353516),-1
1,1,91037,21038,CurbRamp,NaN,1.0,True,g0kskJSQVRkZXwH8mmA3gA,1,0,...,34.062500,-35.000000,99.724670,-2.702782,0.0,0.0,-65.662170,-32.297218,POINT (-122.31624603271484 47.55794906616211),1
2,1,93842,22195,CurbRamp,1.0,1.0,True,npPLy6aGDOcUNgSXA4sq8Q,1,0,...,175.187500,-19.250000,299.396454,2.216354,0.0,0.0,-124.208954,-21.466354,POINT (-122.32888793945312 47.5999755859375),1
3,1,190528,47238,CurbRamp,1.0,1.0,True,KIOwpToFQQi4QK6hxq2Jjg,1,0,...,31.437500,-16.187500,132.761383,-1.222160,0.0,0.0,-101.323883,-14.965340,POINT (-122.3386001586914 47.6960334777832),1
4,1,190672,47291,CurbRamp,1.0,1.0,True,00WCIfeoGC33NDp8s0_KGg,1,0,...,335.875000,-15.803572,180.815201,0.315163,0.0,0.0,155.059799,-16.118734,POINT (-122.28343200683594 47.546104431152344),0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13132,1,80049,16991,CurbRamp,1.0,1.0,True,rjH-Z_Ba6fC6JazX7WCkdw,1,0,...,94.750000,-18.437500,178.104111,2.727684,0.0,0.0,-83.354111,-21.165184,POINT (-122.28788757324219 47.527854919433594),-1
13133,1,80051,16991,CurbRamp,1.0,1.0,True,rjH-Z_Ba6fC6JazX7WCkdw,1,0,...,13.187500,-8.375000,178.104111,2.727684,0.0,0.0,-164.916611,-11.102684,POINT (-122.28797149658203 47.527957916259766),-1
13134,1,82303,17700,CurbRamp,3.0,NaN,True,eYuHkS8M4C_6rWOXE1MAOw,1,1,...,190.875000,-31.625000,75.978729,-1.196411,0.0,0.0,114.896271,-30.428589,POINT (-122.29849243164062 47.54031753540039),0
13135,1,83398,18083,CurbRamp,NaN,1.0,False,9HbBPwLAbnJ57f3Iv0Sl9A,1,0,...,93.035713,-17.678572,201.913620,0.383842,0.0,0.0,-108.877907,-18.062414,POINT (-122.3752212524414 47.530181884765625),-1


In [26]:
df_train

,audit_task_id,label_id,label_type,severity,correct,high_quality_user,gsv_panorama_id,agree_count,disagree_count,notsure_count,...,heading,pitch,photographer_heading,photographer_pitch,label_tags,label_description,heading_diff,pitch_diff,geometry,clustered
0,18604,85055,CurbRamp,1.0,NaN,False,NjPAkwTxWsayAq3kCugvdA,0,0,0,...,107.750000,-15.62500,270.556610,0.907036,0.0,0.0,-162.806610,-16.532036,POINT (-122.3327865600586 47.61859893798828),-1
1,18608,85057,Obstacle,2.0,1.0,False,f_Mibt-6nJU7HTN7kx5ZGQ,2,0,0,...,38.375000,-21.87500,164.133438,0.335564,0.0,0.0,-125.758438,-22.210564,POINT (-122.3318862915039 47.61780548095703),-1
2,18608,85059,CurbRamp,1.0,NaN,False,f_Mibt-6nJU7HTN7kx5ZGQ,0,0,0,...,131.187500,-24.50000,164.133438,0.335564,0.0,0.0,-32.945938,-24.835564,POINT (-122.33183288574219 47.61751174926758),-1
3,18608,85060,CurbRamp,1.0,NaN,False,f_Mibt-6nJU7HTN7kx5ZGQ,0,0,0,...,131.187500,-24.50000,164.133438,0.335564,0.0,0.0,-32.945938,-24.835564,POINT (-122.33187866210938 47.61751937866211),-1
4,18608,85062,CurbRamp,1.0,NaN,False,f_Mibt-6nJU7HTN7kx5ZGQ,0,0,0,...,257.187500,-35.00000,164.133438,0.335564,0.0,0.0,93.054062,-35.335564,POINT (-122.33204650878906 47.617637634277344),-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196809,76181,231270,CurbRamp,1.0,NaN,True,R-sTl5O6sOdVPurgTWy3EQ,0,0,0,...,332.468750,-35.00000,215.130295,-0.157570,0.0,0.0,117.338455,-34.842430,POINT (-122.26725769042969 47.540771484375),-1
196810,76183,231285,NoSidewalk,5.0,NaN,True,N5MrAAtHZO2JGXUrYAYnSQ,0,0,0,...,0.890625,-11.65625,179.161057,-5.825714,0.0,0.0,-178.270432,-5.830536,POINT (-122.26344299316406 47.55229187011719),-1
196811,76193,231338,CurbRamp,1.0,NaN,True,d3oBrJ-nDtKMwlE7svc6pg,0,0,1,...,253.562500,-35.00000,177.959915,-0.962440,0.0,0.0,75.602585,-34.037560,POINT (-122.28179168701172 47.563270568847656),-1
196812,76193,231339,CurbRamp,1.0,NaN,True,AOIBJ12BjFanDJF1Ark05Q,0,0,0,...,110.312500,-35.00000,179.365723,-1.093400,0.0,0.0,-69.053223,-33.906600,POINT (-122.28165435791016 47.56346893310547),-1


In [27]:
# plot the 